In [4]:
from src.evaluation_and_conversion import painted_ply_to_label_dict, compare_result_txt_to_groundtruth_label_dict

import os
import csv

In [5]:
folder = "../../Data/artefact_31_test/"

In [6]:
with open(folder + "/results/best_results.txt", "w") as fresults:
    fresults.write("ObjectID BestResult BestParams\n")
    
    for dir_name in os.listdir(folder+"results/"):
        obj_id = dir_name.split("_r1.00_n4_v256.volume", 1)[0]
        gt_filepath = folder+"groundtruths/" + obj_id + "_c.ply"
        result_files = []
        
        if dir_name != "best_results.txt":
            for filename in os.listdir(folder+"results/"+dir_name+"/"):

                if filename.endswith(".txt") and not filename.endswith("_correct.txt") and not filename.endswith("_timings.txt"):
                    result_files.append(os.path.join(folder+"results/"+dir_name+"/", filename))
                else:
                    continue

            gt_label = painted_ply_to_label_dict(gt_filepath)

            best_result = 0
            best_param = None

            result_table = []
            #result_table.append(["Persistence", "High_thresh", "Low_thresh", "Merge_thresh", "Correctness"])
            for file in result_files:
                correctness, high_thr, low_thr, merge_thr = compare_result_txt_to_groundtruth_label_dict(file, gt_label, metric="IoU", plot_correctness_mask=False)
                result_table.append([high_thr, low_thr, merge_thr, correctness])
                if correctness > best_result:
                    best_result = correctness
                    best_param = [high_thr, low_thr, merge_thr]

            fresults.write(str(obj_id) + " " + str(best_result) + " " + str(best_param) + "\n")

            with open(folder+"results/"+dir_name+"/"+str(obj_id)+"_result_table.csv", "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerows(result_table)

painted_ply_to_label_dict took 9.67923 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.21520 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.58528 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.99314 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.82235 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.91558 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.95483 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.15984 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.19427 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.14374 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.42737 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.16907 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 1.93994 seconds to execute!
compa